In [2]:
import pkg_resources
# Install packages
package_names = ['keras_vggface', 'keras_applications', 'batch_face']
for package_name in package_names:
  try:
    dist = pkg_resources.get_distribution(package_name)
    print(f"{package_name} is already installed with version {dist.version}")
  except pkg_resources.DistributionNotFound:
    print(f"{package_name} is not installed, installing now...")
    !pip install {package_name}

# !pip install --upgrade --force-reinstall keras_vggface
# !pip install keras_vggface
# !pip install keras_applications
# !pip install batch_face

# Fix package files
!sed -i 's/from keras.utils import layer_utils/from tensorflow.python.keras.utils import layer_utils/' /usr/local/lib/python3.10/dist-packages/keras_vggface/models.py
!sed -i 's/from keras.utils.data_utils import get_file/from tensorflow.python.keras.utils.data_utils import get_file/' /usr/local/lib/python3.10/dist-packages/keras_vggface/models.py
!sed -i 's/from keras.utils.data_utils import get_file/from tensorflow.python.keras.utils.data_utils import get_file/' /usr/local/lib/python3.10/dist-packages/keras_vggface/utils.py
!sed -i 's/from keras.engine.topology import get_source_inputs/from tensorflow.python.keras.utils.layer_utils import get_source_inputs/' /usr/local/lib/python3.10/dist-packages/keras_vggface/models.py

# import argparse
import numpy as np
import pandas as pd
import os
import time
from scipy import stats
# import pickle
import sys

expected_directory = '/content/notebook_modules'

# Check if the expected directory or file exists
if not os.path.exists(expected_directory):
    print("Modules directory does not exist, extracting the ZIP file...")
    # Extract the ZIP file since the directory doesn't exist
    !unzip '/content/notebook_modules.zip' -d /content/
else:
    print("Modules directory already exists. No need to extract the ZIP file.")

sys.path.append('/content/notebook_modules')

import sequences
import get_face_areas
from get_models import load_weights_EE, load_weights_LSTM
import run_functions

# from select_video_subset import select_video_subset

import warnings
warnings.filterwarnings('ignore', category = FutureWarning)

# Mount drive
from google.colab import drive

def is_drive_mounted():
    drive_path = '/content/drive'
    return os.path.isdir(drive_path) and os.listdir(drive_path)

# Mount Google Drive if it is not already mounted
if not is_drive_mounted():
    drive.mount('/content/drive')
    print("Google Drive is mounted now!")
else:
    print("Google Drive is already mounted.")

# Test runs are saved in Google Drive in folders named Run_1, Run_2, etc.
# Check the number of the latest run folder
run_folders = [folder for folder in os.listdir('/content/drive/MyDrive/Thesis_Data/CREMA_runs') if 'Run_' in folder]

continue_from_checkpoint = False

# If run folders is empty, this is the first run
# Check if run folders is empty
if not run_folders:
    this_run = 1
    this_run_folder = f'/content/drive/MyDrive/Thesis_Data/CREMA_runs/Run_{this_run}'
    os.makedirs(this_run_folder)
    # Create subfolder for checkpoints
    checkpoint_folder = os.path.join(this_run_folder, 'checkpoints')
    os.makedirs(checkpoint_folder)
elif continue_from_checkpoint:
    latest_run = max([int(folder.split('_')[1]) for folder in run_folders])
    this_run = latest_run
    checkpoint_files = [csv_file for csv_file in os.listdir(f'/content/drive/MyDrive/Thesis_Data/CREMA_runs/Run_{this_run}/checkpoints')]
    last_checkpoint = max([int(checkpoint_file.split("_")[-1].split(".")[0]) for checkpoint_file in checkpoint_files])
    print(f"Continuing from checkpoint {last_checkpoint}.")
else:
    latest_run = max([int(folder.split('_')[1]) for folder in run_folders])
    this_run = latest_run + 1
    this_run_folder = f'/content/drive/MyDrive/Thesis_Data/CREMA_runs/Run_{this_run}'
    os.makedirs(this_run_folder)
    # Create subfolder for checkpoints
    checkpoint_folder = os.path.join(this_run_folder, 'checkpoints')
    os.makedirs(checkpoint_folder)

# Data folder
data_folder = '/content/VideoMP4'

# Extract zip folder on Google Drive and save it in Google Colab workspace
if not os.path.exists(data_folder):
    print("Data folder does not exist, extracting the ZIP file...")
    # Extract the ZIP file since the directory doesn't exist
    !unzip '/content/drive/MyDrive/Thesis_Data/CREMA_runs/video_files.zip' -d /content/
else:
    print("Data folder already exists. No need to extract the ZIP file.")

backbone_model_path = '/content/drive/MyDrive/Thesis_Data/CREMA_runs/Pretrained_models/Backbone_models/weights_0_66_37_wo_gl.h5'
LSTM_model_path = '/content/drive/MyDrive/Thesis_Data/CREMA_runs/Pretrained_models/LSTM_models/CREMA-D_with_config.h5'

emotion_dict = {"NEU": "Neutral", "HAP": "Happiness", "SAD": "Sadness", "SUR": "Surprise", "FEA": "Fear", "DIS": "Disgust", "ANG": "Anger"}

# Create a list of all video files in the data folder if file ends with .mp4
video_files = [file for file in os.listdir(data_folder) if file.endswith(".mp4")]
# print("video_files: ", video_files)
# print()

data = pd.DataFrame(columns=['filename', 'emotion', 'predicted', 'argmax', 'checkpoint'])
data['filename'] = video_files
data['emotion'] = [emotion_dict[video_file.split("_")[2]] for video_file in video_files]

if continue_from_checkpoint:
    video_files = video_files[last_checkpoint:]
    progress = last_checkpoint + 1
else:
    progress = 1

checkpoint_row = 1
for video_file in video_files:
    print(f"Processing video {progress}/{len(video_files)}")
    # Get emotion from substring in file name
    true_emotion = emotion_dict[video_file.split("_")[2]]
    predicted_emotion, predicted_argmax = run_functions.pred_one_video(data_folder, video_file, backbone_model_path, LSTM_model_path)
    # predicted_emotion, predicted_argmax = run_functions.pred_one_video(test_video_path, test_true_label, backbone_model_path, LSTM_model_path)
    data.loc[data['filename'] == video_file, 'predicted'] = predicted_emotion
    data.loc[data['filename'] == video_file, 'argmax'] = predicted_argmax
    data.loc[data['filename'] == video_file, 'checkpoint'] = progress
    if progress % 100 == 0:
        # save the DataFrame to a csv file
        checkpoint_data = data.head(checkpoint_row)
        checkpoint_data.to_csv(os.path.join(checkpoint_folder,f'run_{this_run}_predicted_checkpoint_{progress}.csv'), index=False)
        print(f"Checkpoint {progress} saved.")
    progress += 1
    checkpoint_row += 1

# save the DataFrame to a csv file
data.to_csv(os.path.join(this_run_folder,f'run_{this_run}_predicted.csv'), index=False)
data.head()




keras_vggface is already installed with version 0.6
keras_applications is already installed with version 1.0.8
batch_face is already installed with version 1.4.0
Directory already exists. No need to extract the ZIP file.
Google Drive is already mounted.
drive  face_areas.pkl  notebook_modules  notebook_modules.zip  sample_data
['RetinaFace', 'VideoCamera', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'cv2', 'img_to_array', 'np', 'os', 'utils']
['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'get_face_areas', 'load_weights_EE', 'load_weights_LSTM', 'np', 'pickle', 'pred_one_video', 'sequences', 'stats', 'time', 'warnings']
Name video:  1001_DFA_ANG_XX.mp4
Number total of frames:  68
FPS:  30.0
Video duration: 2.27 s
Frame width: 480
Frame height: 360
Number of frames after sampling:  12
Backbone model:  /content/drive/MyDrive/Thesis_Data/CREMA_runs/Pretrained_models/Ba